# Graduate Rotational Internship Program @The Sparks Foundation
## **Author** : **SOORAJ TS**
### **Task 1** : Object Detection

* *Description : Implement an object detector which identifies the classes of the objects in an image or video*

In [ ]:
#importing opencv and numpy modules
import cv2
import numpy as np

In [ ]:
#declaring threshold
whT = 320

In [ ]:
#loading coco names containing the objects
className = []
with open('coco.names', 'rt') as f:
    classNames = f.read().splitlines()

In [ ]:
#loading the yolo weights and configuration files
model_cfg = 'yolov3.cfg'
model_weights = 'yolov3.weights'

In [ ]:
#setting opencv as backend and target as cpu
net = cv2.dnn.readNet(model_cfg, model_weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
#function for object detection 
def findObjects(output, img):
    hT, wT, cT = img.shape
    boxes = []
    class_ids = []
    confidences = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                w = int(detection[2]*wT)
                h = int(detection[3]*hT)
                x = int((detection[0]*wT)-w/2)
                y = int((detection[1]*hT)-h/2)
                boxes.append([x, y, w, h])
                class_ids.append(class_id)
                confidences.append(float(confidence))

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, nms_threshold=0.3)
    print(indexes)
    
    #setting the color and font to be displayed
    font = cv2.FONT_HERSHEY_SIMPLEX
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))
    
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classNames[class_ids[i]])
        confidence = str(round(confidences[i],2))
        color = colors[i]
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, label + " " + confidence, (x, y+20), font, 0.6, (255, 0, 255), 2)

In [ ]:
#opencv code for loading video
cap = cv2.VideoCapture('Bean video.mp4')
while True :
    success, img = cap.read()
    blob = cv2.dnn.blobFromImage(img, 1/255, (whT, whT), [0, 0, 0], 1, crop= False)
    net.setInput(blob)

    layerNames = net.getLayerNames()
    out_Names = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
    print(out_Names)
    
    outputs = net.forward(out_Names)
    findObjects(outputs, img)

    cv2.imshow('image', img)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()